In [27]:
import folium
import make_catchments

In [21]:
import rasterio
import numpy as np
from rasterio.warp import calculate_default_transform, reproject, Resampling

# Open the source raster
with rasterio.open("Terrain_Melbourne-2.tif") as src:
    # Open the target raster (the one you want to match)
    with rasterio.open("colorado_sample_dem.tiff") as dst:
        # Calculate the transformation parameters
        print(src.crs)
        print(dst.crs)
        transform, width, height = calculate_default_transform(
            src.crs, dst.crs, src.width, src.height, *src.bounds
        )

        # Create an empty array for the reprojected data
        reprojected_data = np.empty((src.count, height, width), dtype=src.dtypes[0])

        # Reproject the data
        reproject(
            source=rasterio.band(src, 1),
            destination=reprojected_data,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=dst.crs,
            resampling=Resampling.nearest  # Choose a suitable resampling method
        )

        # Save the reprojected raster
        with rasterio.open(
            "reprojected_raster.tif",
            "w",
            driver="GTiff",
            width=width,
            height=height,
            count=src.count,
            dtype=src.dtypes[0],
            crs=dst.crs,
            transform=transform,
        ) as dst:
            dst.write(reprojected_data)

EPSG:26917
EPSG:4269


In [22]:
with rasterio.open("reprojected_raster.tif") as src:
    print(src.crs)

EPSG:4269


In [28]:
basins, branches = make_catchments.generate_catchments('Terrain_Melbourne-2.tif',acc_thresh=3000,so_filter=4)


Reading DEM...
Making flow accumulaton map...
Generating stream network...
Calculating pour points...
Generating 184 catchments...
Total runtime is 0.45628139972686765 minutes


c:\Users\alden.summerville\Documents\dev-local\IRL-MIA-development\venv39\lib\site-packages\shapely\measurement.py:44: RuntimeWarning: invalid value encountered in area
  return lib.area(geometry, **kwargs)


In [ ]:
gdf = basins.copy()
print(gdf.is_valid)
print(branches.is_valid)

0      True
2      True
3      True
4      True
6      True
       ... 
220    True
221    True
222    True
224    True
226    True
Length: 184, dtype: bool
0      True
1      True
2      True
3      True
4      True
       ... 
222    True
223    True
224    True
225    True
226    True
Length: 227, dtype: bool


In [31]:
gdf['BasinGeo'] = gdf['BasinGeo'].buffer(0)
branches['geometry'] = branches['geometry'].buffer(0)

In [35]:
gdf = gdf.to_crs(epsg=4269)
branches = branches.to_crs(epsg=4269)

In [36]:
# Visualize output
# gdf = basins.copy()
# separate by stream order
ones = gdf[gdf['Order']==1]
twos = gdf[gdf['Order']==2]
threes = gdf[gdf['Order']==3] # skip fours

# Map 'em!
cols = ['Index','Length','Relief','Order','Slope','AreaSqKm','LocalPP_X','LocalPP_Y','Final_Chain_Val','BasinGeo']
cols_branches = ['Index','Length','Relief','Order','Slope','geometry','LocalPP_X','LocalPP_Y','Final_Chain_Val']

m = branches[cols_branches].explore(color='black')
ones[cols].explore(m=m,color='blue',tiles='Stamen Terrain')
twos[cols].explore(m=m,color='purple',tiles='Stamen Terrain')
threes[cols].explore(m=m,color='yellow',tiles='Stamen Terrain')
folium.LayerControl().add_to(m) 
m

In [26]:
print(gdf.is_valid)
print(branches.is_valid)

0      True
1      True
2      True
3      True
4      True
       ... 
226    True
227    True
228    True
229    True
232    True
Length: 188, dtype: bool
0      True
1      True
2      True
3      True
4      True
       ... 
229    True
230    True
231    True
232    True
233    True
Length: 234, dtype: bool


In [5]:
gdf

,Length,Relief,Order,Slope,Index,LocalPP_X,LocalPP_Y,Profile,Chain,Final_SO,Final_Chain_Val,BasinGeo,AreaSqKm
0,267.162057,2,1,0.428914,0,-80.746265,28.247520,"[25301, 23728, 22156, 20584, 19013, 17442, 158...","[0, 2]",2,2,"POLYGON ((-80.74954 28.24752, -80.74954 28.246...",0.302376
1,205.066472,1,1,0.279399,1,-80.746109,28.247520,"[36341, 34768, 33195, 31622, 30050, 28478, 269...","[1, 2]",2,2,"POLYGON ((-80.74611 28.24752, -80.74611 28.247...",0.237711
2,8.644230,0,2,0.000000,2,-80.746187,28.247599,[4890],[2],2,2,"POLYGON ((-80.74619 28.2476, -80.74619 28.2475...",0.540883
3,276.947767,0,1,0.000000,3,-80.755167,28.247599,"[53507, 51935, 50363, 48791, 47219, 45647, 440...",[3],1,3,"POLYGON ((-80.75517 28.2476, -80.75517 28.2475...",0.217945
4,20.190593,0,1,0.000000,4,-80.733536,28.245100,"[56927, 55356, 53784, 52211, 50638, 49065, 474...","[4, 32]",2,32,"POLYGON ((-80.73354 28.2451, -80.73354 28.2450...",0.228695
...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,369.587823,0,1,0.000000,226,-80.727992,28.163651,"[1676130, 1677703, 1679276, 1680849, 1682422, ...","[226, 231]",2,231,"POLYGON ((-80.72721 28.17021, -80.72721 28.169...",0.265753
227,458.160272,3,1,0.375163,227,-80.655836,28.163573,"[1677037, 1678609, 1680182, 1681754, 1683327, ...","[227, 228]",2,228,"POLYGON ((-80.66005 28.1706, -80.66005 28.1705...",0.289583
228,80.879321,0,2,0.000000,228,-80.655679,28.162948,"[1697520, 1699093, 1700665, 1702237, 1703809, ...",[228],2,228,"POLYGON ((-80.65771 28.17201, -80.65771 28.171...",0.564890
229,509.145487,2,1,0.225065,229,-80.723853,28.166150,"[1679400, 1677827, 1676254, 1674681, 1673108, ...","[229, 230, 231]",2,231,"POLYGON ((-80.71893 28.16912, -80.71893 28.169...",0.271794


In [6]:
import rasterio

# Open the DEM file
with rasterio.open("Terrain_Melbourne-2.tif") as dataset:
    # Get the coordinate reference system (CRS)
    crs = dataset.crs

print(crs)

EPSG:26917
